In [1]:
import os

from transformers import AutoTokenizer, DataCollatorForTokenClassification, AutoTokenizer,\
AutoModelForTokenClassification, AutoModelForMaskedLM, TrainingArguments, Trainer,AutoModelForSequenceClassification,DataCollatorWithPadding
import evaluate
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datasets import load_dataset,Dataset, DatasetDict


f:\Users\mario\.conda\envs\parking\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [124]:
hf_dataset = load_dataset("bgglue/bgglue","examsbg")
hf_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'answerKey', 'info'],
        num_rows: 1512
    })
    validation: Dataset({
        features: ['id', 'question', 'answerKey', 'info'],
        num_rows: 365
    })
    test: Dataset({
        features: ['id', 'question', 'answerKey', 'info'],
        num_rows: 1472
    })
})

In [125]:
hf_dataset["train"][0]

{'id': '21f03ed6-87da-4b97-afcf-d22c39aaf54a',
 'question': {'stem': 'В кой град е сформирано Българското опълчение?',
  'choices': {'text': ['Кишинев', 'Болград', 'Самара', 'Белград'],
   'label': ['A', 'B', 'C', 'D'],
   'para': ['Началото на Българското опълчение е поставено в края на 1876 година в град Кишинев. Много българи се отправят към Кишинев, където се сформира Българското опълчение. В Кишинев Стоян Везенков помага на граф Игнатиев и Михаил Хитрово да сформират Българското опълчение. Председател е на българското дружество „Възраждане“ в Кишинев. С военен обоз Стоян тръгва за Кишинев, за да търси по-малкия си брат Константин, назначен за лекар към Българското опълчение. Опълченец в Българското опълчение. среща своя съчетник Илин Видолов и с неговия билет за свободно придвижване отпътува през Цариград към Кишинев, където 1877 г. постъпва в Българското опълчение. В състава му е Българското опълчение. Столица град Кишинев. Записва се в Българското опълчение. Записва се в Българс

In [126]:
hf_dataset["train"][0]

{'id': '21f03ed6-87da-4b97-afcf-d22c39aaf54a',
 'question': {'stem': 'В кой град е сформирано Българското опълчение?',
  'choices': {'text': ['Кишинев', 'Болград', 'Самара', 'Белград'],
   'label': ['A', 'B', 'C', 'D'],
   'para': ['Началото на Българското опълчение е поставено в края на 1876 година в град Кишинев. Много българи се отправят към Кишинев, където се сформира Българското опълчение. В Кишинев Стоян Везенков помага на граф Игнатиев и Михаил Хитрово да сформират Българското опълчение. Председател е на българското дружество „Възраждане“ в Кишинев. С военен обоз Стоян тръгва за Кишинев, за да търси по-малкия си брат Константин, назначен за лекар към Българското опълчение. Опълченец в Българското опълчение. среща своя съчетник Илин Видолов и с неговия билет за свободно придвижване отпътува през Цариград към Кишинев, където 1877 г. постъпва в Българското опълчение. В състава му е Българското опълчение. Столица град Кишинев. Записва се в Българското опълчение. Записва се в Българс

In [132]:
questions = hf_dataset["train"][0:5]
for i, q in questions.iter:
    print(q)

id
question
answerKey
info


In [112]:
label2id = {'A':0, 'B':1, 'C':2, 'D':3}

In [123]:
def map_answers_to_ids(examples):
    questions = [q for q in examples]
    print(questions)
    # labels = [ q['choices'] for q in examples_questions]
    # answerKeys = [l for l in examples["question"]["choices"]['label']]
    # examples["answer_as_id"] = [label.index(key) for label, key in tuple(labels,answerKeys)]
    return questions

hf_dataset["train"].map(map_answers_to_ids, batched=True)

Map:   0%|          | 0/1512 [00:00<?, ? examples/s]


['id', 'question', 'answerKey', 'info']


TypeError: Provided `function` which is applied to all elements of table returns a variable of type <class 'list'>. Make sure provided `function` returns a variable of type `dict` (or a pyarrow table) to update the dataset or `None` if you are only interested in side effects.

In [26]:
# Load model
model_checkpoint = "mor40/BulBERT-chitanka-model"
model_raw = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=4)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at mor40/BulBERT-chitanka-model and are newly initialized: ['bert.pooler.dense.bias', 'classifier.bias', 'classifier.weight', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
example = hf_dataset['train'][0]

In [79]:
example["answerKey"]

'A'

In [80]:
example["question"]["choices"]['label'].index(example["answerKey"])

0

In [52]:

"".join(example["question"]['choices']['para'])

'Началото на Българското опълчение е поставено в края на 1876 година в град Кишинев. Много българи се отправят към Кишинев, където се сформира Българското опълчение. В Кишинев Стоян Везенков помага на граф Игнатиев и Михаил Хитрово да сформират Българското опълчение. Председател е на българското дружество „Възраждане“ в Кишинев. С военен обоз Стоян тръгва за Кишинев, за да търси по-малкия си брат Константин, назначен за лекар към Българското опълчение. Опълченец в Българското опълчение. среща своя съчетник Илин Видолов и с неговия билет за свободно придвижване отпътува през Цариград към Кишинев, където 1877 г. постъпва в Българското опълчение. В състава му е Българското опълчение. Столица град Кишинев. Записва се в Българското опълчение. Записва се в Българското опълчение. Столицата на републиката е град Кишинев. Командир на Българското опълчение. В сраженията се отличава Българското опълчение. Командир на рота в Българското опълчение. Христо Лекарски постъпва в Българското опълчение. 

In [63]:
question = "В кой град е сформирано Българското опълчение?"
context = """Началото на Българското опълчение е поставено в края на 1876 година в град Кишинев. 
Много българи се отправят към Кишинев, където се сформира Българското опълчение.
В Кишинев Стоян Везенков помага на граф Игнатиев и Михаил Хитрово да сформират Българското опълчение. 
Председател е на българското дружество „Възраждане“ в Кишинев. С военен обоз Стоян тръгва за Кишинев, за да търси по-малкия си брат Константин, 
назначен за лекар към Българското опълчение. Опълченец в Българското опълчение. среща своя съчетник Илин Видолов и с неговия билет за свободно 
придвижване отпътува през Цариград към Кишинев, където 1877 г. постъпва в Българското опълчение. В състава му е Българското опълчение.  """
tokenized_example = tokenizer(example["question"]["stem"], "".join(example["question"]['choices']['para']),return_offsets_mapping=True,return_overflowing_tokens=True, max_length=256,stride=25,truncation="only_second",padding="max_length",)

for idx, window in enumerate(tokenized_example["input_ids"]):
 print(f"Window #{idx} has {len(window)} tokens")

Window #0 has 256 tokens
Window #1 has 256 tokens
Window #2 has 256 tokens
Window #3 has 256 tokens
Window #4 has 256 tokens
Window #5 has 256 tokens
Window #6 has 256 tokens
Window #7 has 256 tokens


In [64]:
for window in tokenized_example["input_ids"]:
 print(f"{tokenizer.decode(window)} \n")

[CLS] в кои град е сформирано българското опълчение? [SEP] началото на българското опълчение е поставено в края на 1876 година в град кишинев. много българи се отправят към кишинев, където се сформира българското опълчение. в кишинев стоян везенков помага на граф игнатиев и михаил хитрово да сформират българското опълчение. председател е на българското дружество „ възраждане “ в кишинев. с военен обоз стоян тръгва за кишинев, за да търси по - малкия си брат константин, назначен за лекар към българското опълчение. опълченец в българското опълчение. среща своя съчетник илин видолов и с неговия билет за свободно придвижване отпътува през цариград към кишинев, където 1877 г. постъпва в българското опълчение. в състава му е българското опълчение. столица град кишинев. записва се в българското опълчение. записва се в българското опълчение. столицата на републиката е град кишинев. командир на българското опълчение. в сраженията се отличава българското опълчение. командир на рота в българското

In [67]:
tokenized_example["offset_mapping"]

[[(0, 0),
  (0, 1),
  (2, 5),
  (6, 10),
  (11, 12),
  (13, 21),
  (21, 23),
  (24, 30),
  (30, 35),
  (36, 40),
  (40, 45),
  (45, 46),
  (0, 0),
  (0, 8),
  (9, 11),
  (12, 18),
  (18, 23),
  (24, 28),
  (28, 33),
  (34, 35),
  (36, 45),
  (46, 47),
  (48, 52),
  (53, 55),
  (56, 58),
  (58, 59),
  (59, 60),
  (61, 67),
  (68, 69),
  (70, 74),
  (75, 77),
  (77, 79),
  (79, 82),
  (82, 83),
  (84, 89),
  (90, 96),
  (96, 97),
  (98, 100),
  (101, 109),
  (110, 113),
  (114, 116),
  (116, 118),
  (118, 121),
  (121, 122),
  (123, 129),
  (130, 132),
  (133, 141),
  (142, 148),
  (148, 153),
  (154, 158),
  (158, 163),
  (163, 164),
  (165, 166),
  (167, 169),
  (169, 171),
  (171, 174),
  (175, 179),
  (179, 180),
  (181, 183),
  (183, 186),
  (186, 189),
  (190, 196),
  (197, 199),
  (200, 204),
  (205, 207),
  (207, 211),
  (211, 213),
  (214, 215),
  (216, 222),
  (223, 228),
  (228, 230),
  (231, 233),
  (234, 242),
  (242, 243),
  (244, 250),
  (250, 255),
  (256, 260),
  (260, 2

In [171]:
def preprocess_function(examples):
    questions = [q["stem"].strip() for q in examples["question"]]
    contexts = [p["choices"]["para"]for p in examples["question"]]
    print(contexts[0])
    inputs = tokenizer(
        questions,
        contexts,
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )
    return inputs

In [175]:
 hf_dataset["train"][:10].map(preprocess_function, batched=True, remove_columns=hf_dataset["train"].column_names)


AttributeError: 'dict' object has no attribute 'map'